In [1]:
from warnings import filterwarnings

filterwarnings('ignore')

In [2]:
from copy import deepcopy
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [5]:
trade_df = pd.read_csv('data/dyadic_trade_3.0.csv', encoding = 'utf8')

trade_df.rename({'flow1': 'money_flow_1', 'flow2': 'money_flow_2',
                 'ccode1': 'c_code_1', 'ccode2': 'c_code_2',
                 'importer1': 'state_name_1', 'importer2': 'state_name_2'},
                axis = 1, inplace = True)

trade_df['c_code_1'] = trade_df['c_code_1'].astype(int)
trade_df['c_code_2'] = trade_df['c_code_2'].astype(int)

trade_df['money_flow_1'] = ([s * 1000000 for s in trade_df['money_flow_1']])
trade_df['money_flow_2'] = ([s * 1000000 for s in trade_df['money_flow_2']])

trade_df.loc[trade_df['money_flow_1'] == -9000000, 'money_flow_1'] = None
trade_df.loc[trade_df['money_flow_2'] == -9000000, 'money_flow_2'] = None

trade_df = trade_df.sort_values(by = 'year', ascending = True).reset_index()

trade_df.drop(['source1', 'source2', 'bel_lux_alt_flow1', 'bel_lux_alt_flow2', 'china_alt_flow1', 'china_alt_flow2', 'version', 'index'], axis = 1, inplace = True)

In [6]:
trade_df_copy = deepcopy(trade_df)

trade_df_copy.rename({'c_code_1': 'c_code_1_copy', 'c_code_2':'c_code_2_copy',
                 'state_name_1': 'state_name_1_copy', 'state_name_2': 'state_name_2_copy',
                 'money_flow_1': 'money_flow_1_copy', 'money_flow_2': 'money_flow_2_copy'},
                axis = 1, inplace = True)

trade_df_copy.rename({'c_code_1_copy': 'c_code_2', 'c_code_2_copy':'c_code_1',
                 'state_name_1_copy': 'state_name_2', 'state_name_2_copy': 'state_name_1',
                 'money_flow_1_copy': 'money_flow_2', 'money_flow_2_copy': 'money_flow_1'},
                axis = 1, inplace = True)

In [7]:
trade_df = pd.concat([trade_df, trade_df_copy], axis = 0)

trade_df = trade_df.sort_values(by = 'year', ascending = True).reset_index()

trade_df.drop(['index'], axis = 1, inplace = True)

In [8]:
trade_df.head()

,c_code_1,c_code_2,money_flow_1,money_flow_2,state_name_1,state_name_2,year
0,210,390,nan,nan,Netherlands,Denmark,1870
1,255,160,190000.00,nan,Germany,Argentina,1870
2,200,70,1450000.00,5110000.00,United Kingdom,Mexico,1870
3,600,255,nan,nan,Morocco,Germany,1870
4,350,135,nan,nan,Greece,Peru,1870


In [9]:
aggregations = {
    'c_code_2': 'count',
    'money_flow_1': 'sum',
    'money_flow_2': 'sum'
    }

trade_df_group = trade_df.groupby(['c_code_1', 'state_name_1', 'year']).agg(aggregations).reset_index()

trade_df_group.rename({'c_code_2': 'num_trade_states', 'money_flow_1': 'export_dollars', 'money_flow_2': 'import_dollars'}, axis = 1, inplace = True)

In [10]:
trade_df_group.to_pickle('pickle/trade_df_group.pkl')

In [11]:
trade_df_group.head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars
0,2,United States of America,1870,36,256170000.00,359400000.00
1,2,United States of America,1871,35,319960000.00,430510000.00
2,2,United States of America,1872,32,313530000.00,353040000.00
3,2,United States of America,1873,32,392890000.00,542810000.00
4,2,United States of America,1874,32,317460000.00,553420000.00


In [12]:
mat_cap_df = pd.read_csv('data/NMC_5_0-wsupplementary.csv', encoding = 'latin-1')

mat_cap_df.rename({'milex': 'military_expenditure', 'milper': 'military_personnel',
                   'irst': 'iron_steel_prod', 'pec': 'prim_energy_consumption',
                   'tpop': 'total_pop', 'upop': 'urban_pop',
                   'upopgrowth': 'urban_pop_growth_rate', 'ccode': 'c_code_1',
                   'statenme': 'state_name_1', 'cinc': 'cinc_score'}, axis = 1, inplace = True)

mat_cap_df['military_expenditure'] = ([s * 1000 for s in mat_cap_df['military_expenditure']])
mat_cap_df['military_personnel'] = ([s * 1000 for s in mat_cap_df['military_personnel']])
mat_cap_df['total_pop'] = ([s * 1000 for s in mat_cap_df['total_pop']])
mat_cap_df['urban_pop'] = ([s * 1000 for s in mat_cap_df['urban_pop']])
mat_cap_df['iron_steel_prod'] = ([s * 2000000 for s in mat_cap_df['iron_steel_prod']])
mat_cap_df['prim_energy_consumption'] = ([s * 2000000 for s in mat_cap_df['prim_energy_consumption']])

mat_cap_df.loc[mat_cap_df['military_expenditure'] == -9000, 'military_expenditure'] = None
mat_cap_df.loc[mat_cap_df['military_personnel'] == -9000, 'military_personnel'] = None
mat_cap_df.loc[mat_cap_df['total_pop'] == -9000, 'total_pop'] = None
mat_cap_df.loc[mat_cap_df['urban_pop'] == -9000, 'urban_pop'] = None
mat_cap_df.loc[mat_cap_df['iron_steel_prod'] == -18000000 , 'iron_steel_prod'] = None
mat_cap_df.loc[mat_cap_df['prim_energy_consumption'] == -18000000 , 'prim_energy_consumption'] = None

mat_cap_df = mat_cap_df.sort_values(by = 'year', ascending = True).reset_index()

mat_cap_df.drop(['stateabb', 'milpersource', 'milpernote', 'milexsource', 'milexnote',
                 'irstsource', 'irstnote', 'pecsource', 'pecnote', 'pecnote',
                 'tpopsource', 'tpopnote', 'upopsource', 'upopnote', 'upopgrowthsource',
                 'irstqualitycode', 'irstanomalycode', 'pecqualitycode', 'pecanomalycode',
                 'tpopqualitycode', 'tpopanomalycode', 'upopqualitycode', 'upopanomalycode',
                 'version', 'urban_pop_growth_rate', 'index'], axis = 1, inplace = True)

In [13]:
mat_cap_df.to_pickle('pickle/mat_cap_df.pkl')

In [14]:
mat_cap_df.head()

,state_name_1,c_code_1,year,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score
0,United States of America,2,1816,3823000.00,17000.00,160000000.00,508000000.00,8659000.00,101000.00,0.04
1,Spain,230,1816,6512000.00,125000.00,20000000.00,0.00,11073000.00,221000.00,0.05
2,Netherlands,210,1816,2375000.00,26000.00,100000000.00,2284000000.00,5610000.00,337000.00,0.04
3,Portugal,235,1816,nan,22000.00,0.00,0.00,2746000.00,179000.00,0.01
4,United Kingdom,200,1816,16942000.00,255000.00,540000000.00,45056000000.00,19520000.00,1957000.00,0.34


In [15]:
alliance_df = pd.read_csv('data/alliance_v4.1_by_directed_yearly.csv', encoding = 'utf8')

alliance_df.rename({'left_censor': 'pre_1816_alliance', 'right_censor': 'in_effect_1231_2012',
                    'neutrality': 'neutrality_treaty', 'nonaggression': 'nonaggression_treaty',
                    'defense': 'defense_treaty', 'entente': 'entente_treaty',
                    'ccode1': 'c_code_1', 'ccode2': 'c_code_2', 'state_name1': 'state_name_1',
                    'state_name2': 'state_name_2'}, axis = 1, inplace = True)

alliance_df['alliance_date_start'] = pd.to_datetime((alliance_df['dyad_st_year']*10000+alliance_df['dyad_st_month']*100+alliance_df['dyad_st_day']).apply(str),format='%Y%m%d')
alliance_df['alliance_date_end'] = pd.to_datetime((alliance_df['dyad_end_year']*10000+alliance_df['dyad_end_month']*100+alliance_df['dyad_end_day']).apply(str),format='%Y%m%d')

alliance_df = alliance_df.sort_values(by = 'year', ascending = True).reset_index()

alliance_df.drop(['dyad_st_day', 'dyad_st_month', 'dyad_st_year', 'dyad_end_day',
                  'dyad_end_month', 'dyad_end_year', 'version4id', 'version', 'index'],
                 axis = 1, inplace = True)

In [16]:
alliance_df.head()

,c_code_1,state_name_1,c_code_2,state_name_2,pre_1816_alliance,in_effect_1231_2012,defense_treaty,neutrality_treaty,nonaggression_treaty,entente_treaty,year,alliance_date_start,alliance_date_end
0,200,United Kingdom,235,Portugal,1,1,1,0,1.00,0.00,1816,1816-01-01,NaT
1,275,Hesse Grand Ducal,300,Austria-Hungary,1,0,1,0,1.00,1.00,1816,1816-01-01,1848-03-15
2,275,Hesse Grand Ducal,273,Hesse Electoral,1,0,1,0,1.00,1.00,1816,1816-01-01,1848-03-15
3,275,Hesse Grand Ducal,271,Wuerttemburg,1,0,1,0,1.00,1.00,1816,1816-01-01,1848-03-15
4,275,Hesse Grand Ducal,269,Saxony,1,0,1,0,1.00,1.00,1816,1816-01-01,1848-03-15


In [17]:
aggregations = {
    'c_code_2': 'count',
    'pre_1816_alliance': 'sum',
    'in_effect_1231_2012': 'sum',
    'defense_treaty': 'sum',
    'neutrality_treaty': 'sum',
    'nonaggression_treaty': 'sum',
    'entente_treaty': 'sum'
    }

alliance_df_group = alliance_df.groupby(['c_code_1', 'state_name_1', 'year']).agg(aggregations).reset_index()

alliance_df_group = alliance_df_group.sort_values(by = 'year', ascending = True).reset_index()

alliance_df_group.rename({'c_code_2': 'num_alliances', 'pre_1816_alliance': 'pre_1816_alliances',
                          'in_effect_1231_2012': 'num_in_effect_1231_2012',
                          'defense_treaty': 'defense_treaties',
                          'neutrality_treaty': 'neutrality_treaties',
                          'nonaggression_treaty': 'nonaggression_treaties',
                          'entente_treaty': 'entente_treaties'}, axis = 1, inplace = True)

alliance_df_group.drop(['index'], axis = 1, inplace = True)

In [18]:
alliance_df_group.to_pickle('pickle/alliance_df_group.pkl')

In [19]:
alliance_df_group.head()

,c_code_1,state_name_1,year,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties
0,230,Spain,1816,1,0,0,1,0,0.00,0.00
1,255,Germany,1816,10,10,0,10,0,7.00,10.00
2,267,Baden,1816,7,7,0,7,0,7.00,7.00
3,271,Wuerttemburg,1816,7,7,0,7,0,7.00,7.00
4,380,Sweden,1816,1,1,0,0,0,0.00,1.00


In [20]:
contiguity_df = pd.read_csv('data/contdird.csv', encoding = 'utf8')

contiguity_df.rename({'state1no': 'c_code_1', 'state2no': 'c_code_2',
                      'state1ab': 'state_name_abb_1', 'state2ab': 'state_name_abb_2',
                      'conttype': 'contiguity_type'}, axis = 1, inplace = True)
                 
contiguity_df.drop(['dyad', 'version'], axis = 1, inplace = True)

In [21]:
contiguity_df.to_pickle('pickle/contiguity_df.pkl')

In [22]:
contiguity_df.head()

,c_code_1,state_name_abb_1,c_code_2,state_name_abb_2,year,contiguity_type
0,2,USA,20,CAN,1920,1
1,2,USA,20,CAN,1921,1
2,2,USA,20,CAN,1922,1
3,2,USA,20,CAN,1923,1
4,2,USA,20,CAN,1924,1


In [23]:
mid_df = pd.read_csv('data/dyadic MIDs 3.1.csv', encoding = 'utf8')
    
mid_df.rename({'revstata': 'revision_sought', 'revtypea': 'revision_type',
               'fatleva': 'state_fatality_bin', 'highmcaa': 'highest_mca',
               'hihosta': 'highest_hostility', 'durindx': 'year_num',
               'midc3hia': 'highest_action_for_state', 'orignata': 'origin_participant',
               'notarg': 'num_against', 'settlmnt': 'settlement_type',
               'fatlev': 'total_fatality_bin', 'noinit': 'num_with',
               'disno': 'dispute_id', 'statea': 'c_code_1', 'stateb': 'c_code_2',
               'namea': 'state_name_abb_a', 'nameb': 'state_name_abb_b',
               'cumdurat': 'cumulative_duration', 'ongo2010': 'ongoing_2010'}, axis = 1, inplace = True)

mid_df['date_start'] = pd.to_datetime((mid_df['strtyr']*10000+mid_df['strtmnth']*100+mid_df['strtday']).apply(str),format='%Y%m%d')
mid_df['date_end'] = pd.to_datetime((mid_df['endyear']*10000+mid_df['endmnth']*100+mid_df['endday']).apply(str),format='%Y%m%d')

mid_df.loc[mid_df['sideaa'] == 1, 'initiator'] = 1
mid_df.loc[mid_df['sideaa'] != 1, 'initiator'] = 0

mid_df = mid_df.sort_values(by = 'year', ascending = True).reset_index()

mid_df.drop(['strtmnth', 'strtday', 'strtyr', 'endmnth', 'endday',
             'endyear', 'dyindex', 'duration', 'disno4', 'sideaa',
             'sideab', 'revstatb', 'revtypeb', 'fatlevb', 'highmcab',
             'hihostb', 'hihost', 'orignatb', 'recip', 'rolea', 'roleb',
             'mid3hiact', 'mid3hib', 'change', 'changetype_1',
             'changetype_2', 'highact', 'new', 'mid3hia', 'index'],
            axis = 1, inplace = True)

In [24]:
mid_df.head()

,dispute_id,c_code_1,state_name_abb_a,c_code_2,state_name_abb_b,year,outcome,settlement_type,total_fatality_bin,num_with,num_against,revision_sought,revision_type,state_fatality_bin,highest_mca,highest_hostility,origin_participant,war,year_num,cumulative_duration,ongoing_2010,date_start,date_end,initiator
0,3239,230,SPN,2,USA,1816,2,3,0,1,1,0,0,0,1,1,1,0,1,1,0,1816-07-27,1816-07-27,0.00
1,3239,2,USA,230,SPN,1816,1,3,0,1,1,1,2,0,19,4,1,0,1,1,0,1816-07-27,1816-07-27,1.00
2,3321,365,RUS,640,TUR,1817,4,1,0,1,1,1,1,0,7,3,1,0,1,51,0,1817-09-12,1817-11-01,1.00
3,3321,640,TUR,365,RUS,1817,3,1,0,1,1,0,0,0,13,3,1,0,1,51,0,1817-09-12,1817-11-01,0.00
4,1567,230,SPN,2,USA,1818,2,1,-9,1,1,0,0,-9,19,4,1,0,1,77,0,1818-03-15,1818-05-30,0.00


In [25]:
aggregations = {
    'c_code_2': 'count',
    'num_with': 'sum',
    'num_against': 'sum',
    'cumulative_duration': 'sum',
    'war': 'sum',
    'origin_participant': 'mean',
    'ongoing_2010': 'mean',
    'revision_sought': 'mean',
    'revision_type': pd.Series.mode,
    'state_fatality_bin': pd.Series.mode,
    'outcome': pd.Series.mode,
    'settlement_type': pd.Series.mode,
    'highest_mca': pd.Series.mode,
    'highest_hostility': pd.Series.mode
    }

mid_df_group = mid_df.groupby(['c_code_1', 'state_name_abb_a', 'year']).agg(aggregations).reset_index()

for i, country in enumerate(mid_df_group['c_code_1']):
    mid_df_group.loc[i, 'num_with'] = mid_df_group['num_with'][i]/mid_df_group['c_code_2'][i]
    mid_df_group.loc[i, 'num_against'] = mid_df_group['num_against'][i]/mid_df_group['c_code_2'][i]
    mid_df_group.loc[i, 'cumulative_duration'] = mid_df_group['cumulative_duration'][i]/mid_df_group['c_code_2'][i]
    mid_df_group.loc[i, 'cumulative_duration'] = mid_df_group['cumulative_duration'][i]/365

mid_df_group = mid_df_group.sort_values(by = 'year', ascending = True).reset_index()

mid_df_group.rename({'c_code_2': 'num_conflicts', 'num_with': 'avg_with', 'num_against': 'avg_against',
                     'cumulative_duration': 'avg_cum_duration', 'war': 'num_wars',
                     'origin_participant': 'origin_participant_pct', 'revision_sought': 'revision_pct',
                    'revision_type': 'revision_type_mode', 'state_fatality_bin': 'state_fatality_bin_mode',
                     'outcome': 'outcome_mode', 'settlement_type': 'settlement_type_mode',
                     'highest_mca': 'highest_mca_mode', 'highest_hostility': 'highest_hostility_mode'}, axis = 1, inplace = True)

mid_df_group.drop(['index'], axis = 1, inplace = True)

In [26]:
mid_df_group.to_pickle('pickle/mid_df_group.pkl')

In [27]:
mid_df_group.head()

,c_code_1,state_name_abb_a,year,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode
0,2,USA,1816,1,1.00,1.00,0.00,0,1.00,0.00,1.00,2,0,1,3,19,4
1,230,SPN,1816,1,1.00,1.00,0.00,0,1.00,0.00,0.00,0,0,2,3,1,1
2,365,RUS,1817,1,1.00,1.00,0.14,0,1.00,0.00,1.00,1,0,4,1,7,3
3,640,TUR,1817,1,1.00,1.00,0.14,0,1.00,0.00,0.00,0,0,3,1,13,3
4,2,USA,1818,1,1.00,1.00,0.21,0,1.00,0.00,1.00,1,-9,1,1,16,4


In [28]:
gov_df = pd.read_csv('data/ddrevisited_data_v1.csv', encoding = 'latin-1')

gov_df.rename({'chgterr': 'territory_change', 'ychgterr': 'territory_change_year',
               'entryy': 'first_recorded_year', 'exity': 'last_recorded_year',
               'bornyear': 'born_year', 'endyear': 'died_year',
               'exselec': 'election_type', 'legselec': 'legislation_type',
               'closed': 'legislature_status', 'dejure': 'party_legal_status',
               'defacto': 'party_existance', 'defacto2': 'party_existance_outside_regime', 
               'lparty': 'legislature_parties', 'incumb': 'incumbent_type',
               'collect': 'collective_leadership', 'eheads': 'num_leadership_changes',
               'ehead': 'leader_name', 'epost': 'post_name', 'edate': 'entrance_date',
               'ageeh': 'leader_tenure', 'emil': 'military_leader',
               'royal': 'royal_leader', 'comm': 'communist_leader',
               'edeath': 'leader_died', 'democracy': 'democratic_regime',
               'assconfid': 'cabinet_assembly', 'poppreselec': 'popular_election',
               'regime': 'regime_type', 'ttd': 'transition_democracy',
               'tta': 'transition_dictatorship', 'agedem': 'age_govt',
               'stra': 'num_transitions_ever', 'cowcode': 'c_code_1',
               'ctryname': 'state_name_1', 'headdiff': 'nominal_vs_eff_diff',
               'cowcode2': 'c_code_2'},
              axis = 1, inplace = True)

gov_df.drop(['order', 'aclpcode', 'c_code_2', 'qogctycode', 'qogctylett',
             'qogctyyear', 'ccdcodelet', 'ccdcodenum', 'aclpyear',
             'cowcode2year', 'cowcodeyear', 'flagc_cowcode2', 'flage_cowcode2',
             'imf_code', 'politycode', 'bankscode', 'dpicode', 'uncode', 'un_region',
             'un_region_name', 'un_continent', 'last_recorded_year',
             'dupcow', 'dupwdi', 'dupun', 'dupdpi', 'dupimf', 'dupbanks',
             'cid', 'wdicode', 'un_continent_name', 'aclp_region', 'type2',
             'nheads', 'nmil', 'nhead', 'npost', 'ndate', 'entrance_date',
             'tenure08', 'ecens08', 'flageh', 'tt', 'flagc', 'flagdem',
             'flagreg', 'agereg', 'first_recorded_year', 'born_year',
             'died_year', 'leader_name', 'post_name'], axis = 1, inplace = True)

In [29]:
gov_df.to_pickle('pickle/gov_df.pkl')

In [30]:
gov_df.head()

,state_name_1,year,c_code_1,territory_change,territory_change_year,election_type,legislation_type,legislature_status,party_legal_status,party_existance,party_existance_outside_regime,legislature_parties,incumbent_type,collective_leadership,num_leadership_changes,leader_tenure,military_leader,royal_leader,nominal_vs_eff_diff,communist_leader,leader_died,democratic_regime,cabinet_assembly,popular_election,regime_type,transition_democracy,transition_dictatorship,age_govt,num_transitions_ever
0,Afghanistan,1946.00,700.00,0.00,0.00,3.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,14.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,5.00,0.00,0.00,18.00,0.00
1,Afghanistan,1947.00,700.00,0.00,0.00,3.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,15.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,5.00,0.00,0.00,19.00,0.00
2,Afghanistan,1948.00,700.00,0.00,0.00,3.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,5.00,0.00,0.00,20.00,0.00
3,Afghanistan,1949.00,700.00,0.00,0.00,3.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,17.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,5.00,0.00,0.00,21.00,0.00
4,Afghanistan,1950.00,700.00,0.00,0.00,3.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,5.00,0.00,0.00,22.00,0.00


In [31]:
wrp_df = pd.read_csv('data/WRP_national.csv', encoding = 'utf8')

In [32]:
wrp_df.head()

,year,state,name,chrstprot,chrstcat,chrstorth,chrstang,chrstothr,chrstgen,judorth,jdcons,judref,judothr,judgen,islmsun,islmshi,islmibd,islmnat,islmalw,islmahm,islmothr,islmgen,budmah,budthr,budothr,budgen,zorogen,hindgen,sikhgen,shntgen,bahgen,taogen,jaingen,confgen,syncgen,anmgen,nonrelig,othrgen,sumrelig,pop,chrstprotpct,chrstcatpct,chrstorthpct,chrstangpct,chrstothrpct,chrstgenpct,judorthpct,judconspct,judrefpct,judothrpct,judgenpct,islmsunpct,islmshipct,islmibdpct,islmnatpct,islmalwpct,islmahmpct,islmothrpct,islmgenpct,budmahpct,budthrpct,budothrpct,budgenpct,zorogenpct,hindgenpct,sikhgenpct,shntgenpct,bahgenpct,taogenpct,jaingenpct,confgenpct,syncgenpct,anmgenpct,nonreligpct,othrgenpct,sumreligpct,total,dualrelig,datatype,sourcereliab,recreliab,reliabilevel,Version,sourcecode
0,1945,2,USA,66069671,38716742,1121898,2400000,1956807,110265118,821489,1364508,1902885.00,552300,4641182,0,0,0,0,0,0,0.00,0,0,0,1601218,1601218,0,0,0,0,0,0,0,0,0,0,22874544,545938,139382062,139928000,0.47,0.28,0.01,0.02,0.01,0.79,0.01,0.01,0.01,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.16,0.00,1.00,1.00,0,34,2,10,Medium,1.10,13
1,1950,2,USA,73090083,42635882,3045420,3045420,1177214,122994019,1078078,1790706,2497244.00,724809,6090837,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22568130,618022,151652986,152271008,0.48,0.28,0.02,0.02,0.01,0.81,0.01,0.01,0.02,0.00,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.15,0.00,1.00,1.00,0,34,6,28,Low,1.10,18
2,1955,2,USA,79294628,46402368,3454916,2572767,2277091,134001770,944000,1568000,2186666.00,634666,5333332,0,0,0,0,0,0,0.00,0,0,0,90173,90173,0,0,0,0,0,0,0,0,0,0,23303540,3202185,162728815,165931000,0.48,0.28,0.02,0.02,0.01,0.81,0.01,0.01,0.01,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.14,0.02,0.98,1.00,0,134,5,10,Medium,1.10,15
3,1960,2,USA,90692928,50587880,3334535,2710065,2908939,150234347,973500,1617000,2255000.00,654500,5500000,0,0,0,0,0,0,0.00,0,0,0,2012131,2012131,0,0,0,0,0,0,0,0,0,0,21548225,1376297,179294703,180671000,0.50,0.28,0.02,0.01,0.02,0.83,0.01,0.01,0.01,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.12,0.01,0.99,1.00,0,134,2,10,Medium,1.10,13
4,1965,2,USA,94165803,64761783,4792868,2822149,973155,167515758,991200,1646400,2296000.00,666400,5600000,0,0,0,0,0,0,0.00,0,0,0,1080892,1080892,0,0,0,0,0,0,0,0,0,0,19852362,581988,194049012,194631000,0.48,0.33,0.02,0.01,0.01,0.86,0.01,0.01,0.01,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.10,0.00,1.00,1.00,0,134,8,28,Low,1.10,20


In [33]:
one_df = pd.merge(trade_df_group, mat_cap_df, how = 'outer', on = ['c_code_1', 'year'])

one_df.rename({'state_name_1_x': 'state_name_1'}, axis = 1, inplace = True)

one_df.drop(['state_name_1_y'], axis = 1, inplace = True)

In [34]:
one_df.head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score
0,2,United States of America,1870,36.00,256170000.00,359400000.00,13128000.00,50000.00,3384000000.00,75558000000.00,39905000.00,4130000.00,0.10
1,2,United States of America,1871,35.00,319960000.00,430510000.00,11811000.00,42000.00,3470000000.00,79092000000.00,40938000.00,4302000.00,0.10
2,2,United States of America,1872,32.00,313530000.00,353040000.00,14246000.00,42000.00,5180000000.00,96958000000.00,41972000.00,4481000.00,0.12
3,2,United States of America,1873,32.00,392890000.00,542810000.00,15014000.00,43000.00,5204000000.00,107348000000.00,43006000.00,4667000.00,0.12
4,2,United States of America,1874,32.00,317460000.00,553420000.00,12899000.00,44000.00,4878000000.00,103472000000.00,44040000.00,4862000.00,0.11


In [35]:
two_df = pd.merge(one_df, alliance_df_group, how = 'outer', on = ['c_code_1', 'year'])

two_df.rename({'state_name_1_x': 'state_name_1'}, axis = 1, inplace = True)

two_df.drop(['state_name_1_y'], axis = 1, inplace = True)

In [36]:
two_df.head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties
0,2,United States of America,1870,36.00,256170000.00,359400000.00,13128000.00,50000.00,3384000000.00,75558000000.00,39905000.00,4130000.00,0.10,nan,nan,nan,nan,nan,nan,nan
1,2,United States of America,1871,35.00,319960000.00,430510000.00,11811000.00,42000.00,3470000000.00,79092000000.00,40938000.00,4302000.00,0.10,nan,nan,nan,nan,nan,nan,nan
2,2,United States of America,1872,32.00,313530000.00,353040000.00,14246000.00,42000.00,5180000000.00,96958000000.00,41972000.00,4481000.00,0.12,nan,nan,nan,nan,nan,nan,nan
3,2,United States of America,1873,32.00,392890000.00,542810000.00,15014000.00,43000.00,5204000000.00,107348000000.00,43006000.00,4667000.00,0.12,nan,nan,nan,nan,nan,nan,nan
4,2,United States of America,1874,32.00,317460000.00,553420000.00,12899000.00,44000.00,4878000000.00,103472000000.00,44040000.00,4862000.00,0.11,nan,nan,nan,nan,nan,nan,nan


In [37]:
three_df = pd.merge(two_df, mid_df_group, how = 'outer', on = ['c_code_1', 'year'])

three_df.drop(['state_name_abb_a'], axis = 1, inplace = True)

In [38]:
three_df.head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode
0,2,United States of America,1870,36.00,256170000.00,359400000.00,13128000.00,50000.00,3384000000.00,75558000000.00,39905000.00,4130000.00,0.10,nan,nan,nan,nan,nan,nan,nan,2.00,2.00,1.00,0.06,0.00,0.50,0.00,0.00,0,0,4,3,"[8, 19]","[3, 4]"
1,2,United States of America,1871,35.00,319960000.00,430510000.00,11811000.00,42000.00,3470000000.00,79092000000.00,40938000.00,4302000.00,0.10,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2,2,United States of America,1872,32.00,313530000.00,353040000.00,14246000.00,42000.00,5180000000.00,96958000000.00,41972000.00,4481000.00,0.12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
3,2,United States of America,1873,32.00,392890000.00,542810000.00,15014000.00,43000.00,5204000000.00,107348000000.00,43006000.00,4667000.00,0.12,nan,nan,nan,nan,nan,nan,nan,2.00,1.00,1.50,0.07,0.00,1.00,0.00,0.50,"[0, 2]",0,"[3, 5]","[1, 3]","[2, 14]","[2, 3]"
4,2,United States of America,1874,32.00,317460000.00,553420000.00,12899000.00,44000.00,4878000000.00,103472000000.00,44040000.00,4862000.00,0.11,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df = pd.merge(three_df, gov_df, how = 'inner', on = ['c_code_1', 'year'])

df = df.sort_values(by = 'year', ascending = True).reset_index()

df.rename({'state_name_1_x': 'state_name_1'}, axis = 1, inplace = True)

df.drop(['state_name_1_y', 'index'], axis = 1, inplace = True)

In [40]:
df.head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode,territory_change,territory_change_year,election_type,legislation_type,legislature_status,party_legal_status,party_existance,party_existance_outside_regime,legislature_parties,incumbent_type,collective_leadership,num_leadership_changes,leader_tenure,military_leader,royal_leader,nominal_vs_eff_diff,communist_leader,leader_died,democratic_regime,cabinet_assembly,popular_election,regime_type,transition_democracy,transition_dictatorship,age_govt,num_transitions_ever
0,2,United States of America,1946,65.00,160000000.00,14.40,45133984000.00,3030000.00,120842000000.00,2376288000000.00,141389000.00,39725000.00,0.36,19.00,0.00,0.00,19.00,0.00,0.00,19.00,2.00,1.00,1.50,0.27,0.00,0.50,0.00,0.00,0,"[0, 1]",5,3,"[7, 14]",3,0.00,0.00,1.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,2.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,2.00,0.00,0.00,77.00,0.00
1,230,Spain,1946,65.00,14.40,160000000.00,132662000.00,462000.00,1282000000.00,25676000000.00,27020000.00,6164000.00,0.02,1.00,0.00,0.00,0.00,1.00,1.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,3.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,8.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,4.00,0.00,0.00,10.00,1.00
2,235,Portugal,1946,65.00,0.00,0.00,48569000.00,nan,2000000.00,3156000000.00,8120000.00,1036000.00,0.00,2.00,1.00,1.00,1.00,1.00,2.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,1.00,0.00,0.00,0.00,15.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,3.00,0.00,0.00,15.00,1.00
3,290,Poland,1946,65.00,0.00,0.00,2638901000.00,132000.00,2438000000.00,97230000000.00,23958000.00,2411000.00,0.02,2.00,0.00,0.00,2.00,0.00,1.00,1.00,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,3.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,1.00,1.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,3.00,0.00,0.00,2.00,1.00
4,310,Hungary,1946,65.00,10200000.00,11500000.00,105863000.00,18000.00,686000000.00,10648000000.00,9042000.00,1389000.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,1.00,0.00,0.00,2.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,3.00,0.00,0.00,2.00,0.00


In [41]:
df = df[df['year'] >= 1945]

In [42]:
df.loc[df['export_dollars'] == 0, 'export_dollars'] = None
df.loc[df['import_dollars'] == 0, 'import_dollars'] = None
df.loc[df['cinc_score'] == 0, 'cinc_score'] = None

df.loc[df['num_alliances'].isnull(), 'num_alliances'] = 0
df.loc[df['pre_1816_alliances'].isnull(), 'pre_1816_alliances'] = 0
df.loc[df['num_in_effect_1231_2012'].isnull(), 'num_in_effect_1231_2012'] = 0
df.loc[df['defense_treaties'].isnull(), 'defense_treaties'] = 0
df.loc[df['neutrality_treaties'].isnull(), 'neutrality_treaties'] = 0
df.loc[df['nonaggression_treaties'].isnull(), 'nonaggression_treaties'] = 0
df.loc[df['entente_treaties'].isnull(), 'entente_treaties'] = 0
df.loc[df['num_conflicts'].isnull(), 'num_conflicts'] = 0
df.loc[df['avg_cum_duration'].isnull(), 'avg_cum_duration'] = 0
df.loc[df['num_wars'].isnull(), 'num_wars'] = 0
df.loc[df['state_fatality_bin_mode'].isnull(), 'state_fatality_bin_mode'] = 0
df.loc[df['revision_type_mode'].isnull(), 'revision_type_mode'] = 0

df.loc[df['highest_mca_mode'].isnull(), 'highest_mca_mode'] = 1
df.loc[df['highest_hostility_mode'].isnull(), 'highest_hostility_mode'] = 1

# df.loc[df['avg_with'].isnull(), 'avg_with'] = 0
# df.loc[df['avg_against'].isnull(), 'avg_against'] = 0
# df.loc[df['origin_participant_pct'].isnull(), 'origin_participant_pct'] = 0
# df.loc[df['revision_pct'].isnull(), 'revision_pct'] = 0

In [43]:
df[df['state_name_1'].isnull()]

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode,territory_change,territory_change_year,election_type,legislation_type,legislature_status,party_legal_status,party_existance,party_existance_outside_regime,legislature_parties,incumbent_type,collective_leadership,num_leadership_changes,leader_tenure,military_leader,royal_leader,nominal_vs_eff_diff,communist_leader,leader_died,democratic_regime,cabinet_assembly,popular_election,regime_type,transition_democracy,transition_dictatorship,age_govt,num_transitions_ever
420,740,NaN,1951,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,6.00,0.00
636,260,NaN,1954,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,14.00,0.00,0.00,14.00,0.00,14.00,14.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,6.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,6.00,1.00
1099,652,NaN,1959,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,20.00,0.00,14.00,10.00,0.00,18.00,20.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,3.00,0.00,0.00,14.00,0.00
1154,652,NaN,1960,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,20.00,0.00,14.00,10.00,0.00,18.00,20.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,3.00,0.00,0.00,15.00,0.00
2508,771,NaN,1971,nan,nan,nan,nan,nan,nan,nan,70416000.00,2081000.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,3.00,2.00,0.00,2.00,2.00,2.00,0.00,1.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,3.00,0.00,0.00,1.00,0.00
8521,341,NaN,2006,nan,nan,nan,53000000.00,nan,0.00,2838000000.00,617000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,3.00,0.00,0.00,1.00,0.00
8578,341,NaN,2007,nan,nan,nan,59000000.00,nan,348000000.00,2552000000.00,618000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,5.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,3.00,0.00,0.00,2.00,0.00
8762,341,NaN,2008,nan,nan,nan,71000000.00,6000.00,340000000.00,3008000000.00,619000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,6.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,3.00,0.00,0.00,3.00,0.00


In [44]:
len(df['c_code_1'].unique())

198